<a href="https://colab.research.google.com/github/casangi/ngcasa/blob/master/docs/ngcasa_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulation

Radio interferometry data analysis applications and algorithms may be assembled from CNGI and ngCASA building blocks. A user may choose to implement their own analysis scripts, use a pre-packaged task similar to those in current CASA or embed ngCASA and CNGI methods in a production pipeline DAG.

Simulation is an application that uses the simulator, calibrator, flagger and imager modules. Make the MS frame. Calculate all the meta-data and coordinates. 
Use the calibrator and imager modules for the data prediction and corruption.

Note : The API definition should mostly follow the casa6.casatools.simulator interface, except for the steps of calculating visibilities. 

Note : The following examples represent preliminary design ideas that illustrate how ngCASA science applications may be assembled. The API and usage example scripts will change as the details are refined. The current focus is to evaluate whether the CNGI and ngCASA functional design and infrastructure adequately addresses algorithmic needs and how it may be best leveraged for scaleable high performance computing. Questions raised via this initial exercise will guide the design of [future prototypes](https://ngcasa.readthedocs.io/en/latest/ngcasa_development.html#List-of-Future-Prototypes), continued evaluation the chosen infrastructure, and the final function hierarchy and API definition. 

[Edit this notebook in colab](https://colab.research.google.com/github/casangi/ngcasa/blob/master/docs/ngcasa_flagging.ipynb)   : Please note that these examples are currently notional, and not yet runnable in the colab!

## Simulate a wideband mosaic observation with a heterogenous array
This is an ngVLA and ALMA use-case. 

- Call the simulator methods in a sequence to generate an empty dataset containing only coordinates and metadata
- Call flagger methods to flag shadowed or low-elevation data
- Call imager methods to predict visibilities
- Call calibrator methods to corrupt the data

Assumption : A true-sky model image already exists in img_dataset. A cal_dataset also exist (to apply gain errors)

In [0]:
# List of mosaic pointings
list_fields=[{'ra1':'xxxx', 'dec1':'xxxx'},
             {'ra2':'xxxx', 'dec2':'xxxx'}]
# List of spectral windows with pol specs
list_spws=[{'start':'1GHz','nchan':100, 'width'='10MHz', 'pol'='RR,LL'},
           {'start':'2GHz','nchan':1000, 'width'='1MHz', 'pol'='RR,LL'}]
# List of scans
list_scans=[{'date':'xxx', 'ha_start':'-4h', 'ha_stop':'-3h', 'integration'='1s'},
            {'date':'xxx', 'ha_start':'-2h', 'ha_stop':'-1h', 'integration'='1s'} ]

# Construct meta-data for the observation.
sim_dataset = None
for scan in scans:
    for field in fields:
        for spw in spws:
            sim_dataset = ngcasa.simulation.add_meta_data(sim_dataset, scan, field, spw)

# Flag data for low elevation or shadows
ngcasa.flagging.shadow(sim_dataset)
ngcasa.flagging.elevation(sim_dataset)

# Predict model visibilities (in lsrk frame) from an image, and write to the DATA array in the XDS
cngi.vis.regridspw(sim_dataset)  # Convert from topo to lsrk
ngcasa.imaging.make_gridding_convolution_function(img_dataset, gridpars) # Setup de-gridding (include het-array PBs)
ngcasa.imaging.predict_modelvis_image(sim_dataset, component_list, arr_name='DATA') # De-grid and predict model
cngi.vis.regridspw(sim_dataset)  # Convert from lsrk back to data frame (topo)

# Corrupt by antenna gains 
ngcasa.calibration.apply(sim_dataset, cal_dataset_corrupt, use_arr_name='DATA', out_arr_name='DATA')

# Add Gaussian random noise
ngcasa.simulation.add_noise(sim_dataset)

# Save to disk
cngi.write_zarr(sim_dataset)